In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('during2_covidTweets.csv')
print(df.username.unique())
df.head()
df.count()

['nicktompan' 'AndyVermaut' 'eeeer71' 'yasu_yasuno_sa' 'islandofcovid'
 'COVIDLive' 'latimes' 'newsfilterio' 'VonPyotr' 'tazalot1981'
 'Mott_Mason' 'Ravie777' 'AquariusOnFire' 'CTVNews' 'pop_popculture'
 'BobinSea' 'unstoppable8672' 'renztamayo' 'merapimanf' 'TraderMarcoCost'
 'VivekSubbiah' 'mlauriat' 'boatratbown' 'Hibatu3' 'NZStuff'
 'Alan_Nishihara' 'jonathanrockoff' 'NuritBaytch' 'WatchOurCity' 'elvisjj'
 'JerrBoi' 'meer10001' 'ZyiteGadgets' 'bmangh' 'CassieY4' 'usatodaysun21'
 'Su15100307' 'tjmakiboi' 'roto_rua' 'jayjames4' 'chuck85258'
 'SetTheoryVirus' 'Reuters' 'DennisKendel' 'fox5dc' 'TweeterSomebody'
 'Nan10' 'jebender1' 'caa1000' 'Melodys_Musings' 'wbaltv11' 'ErieNewsNow'
 'bornblonde0077' 'MelanieMoore' 'EGirald' 'easyOntario' 'RichinWriterss'
 'SocialistVoice' 'US_FDA' 'Okie_Resister' 'mcbc' 'sapp_erlot'
 'alexselby1770' 'apokerplayer' 'corndnc1' 'openletterbot' '1NewsNZ'
 'AstorAaron' 'DrEricDing' 'KodyKinsley' 'HammerToe' 'helenbinchy'
 'Teelin' 'NedPagliarulo' 'leemors

datetime          58452
tweet_id          58452
text              58452
username          58452
url               58452
user_location     49537
like_count        58452
retweet_count     58452
follower_count    58452
reply_count       58452
verified          58452
hashtags           6867
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['How Japan Has Nearly Wiped Out Covid Infections &amp; Deaths | Natures Real '
 'Health ']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['how', 'japan', 'has', 'nearly', 'wiped', 'out', 'covid', 'infections', 'amp', 'deaths', 'natures', 'real', 'health']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['how', 'japan', 'has_nearly_wiped', 'out', 'covid', 'infections', 'amp', 'deaths', 'natures_real_health']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['nearly_wipe', 'infection', 'death', 'natures_real', 'health']]


In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [15]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

array(['nearly_wiped', 'natures_real', 'microbiome_produces', ...,
       'valid_source', 'technology_networks', 'aldous_huxley'],
      dtype=object)

In [16]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.602*"infection" + 0.109*"symptom" + 0.060*"ease" + 0.038*"aid" + '
  '0.022*"fund" + 0.000*"ecosearch" + 0.000*"die" + 0.000*"look" + '
  '0.000*"month" + 0.000*"even"'),
 (1,
  '0.185*"test" + 0.102*"cause" + 0.074*"positive" + 0.061*"many" + '
  '0.060*"live" + 0.059*"increase" + 0.057*"hit" + 0.037*"keep" + 0.034*"find" '
  '+ 0.027*"measure"'),
 (2,
  '0.277*"people" + 0.141*"day" + 0.101*"booster" + 0.058*"public" + '
  '0.047*"expert" + 0.047*"receive" + 0.040*"warn" + 0.034*"hospitalize" + '
  '0.032*"share" + 0.026*"effect"'),
 (3,
  '0.117*"study" + 0.102*"number" + 0.079*"testing" + 0.073*"school" + '
  '0.063*"world" + 0.045*"much" + 0.044*"also" + 0.036*"outbreak" + '
  '0.035*"try" + 0.035*"treat"'),
 (4,
  '0.000*"dcm" + 0.000*"rulebreaker" + 0.000*"rampage" + 0.000*"vegaization" + '
  '0.000*"utilization" + 0.000*"northern" + 0.000*"msd" + 0.000*"livelihood" + '
  '0.000*"cave" + 0.000*"batu"'),
 (5,
  '0.628*"vaccine" + 0.267*"say" + 0.049*"country" + 0.012*"b

In [18]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




Perplexity:  -17.856803358572087

Coherence Score:  0.3010097902371597


In [19]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\hannah.kim\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.282751 -0.385088       1        1  28.037362
1      0.129823  0.120661       2        1   8.280276
5      0.160201  0.217499       3        1   7.694800
3      0.061624  0.026975       4        1   5.641065
2      0.093022  0.059226       5        1   5.569825
7      0.085586  0.050192       6        1   5.491420
17     0.094734  0.061798       7        1   4.971805
6      0.052084  0.022036       8        1   4.711414
13     0.029217  0.007546       9        1   4.518147
19     0.030942  0.009158      10        1   4.207572
12     0.030909  0.009416      11        1   4.172049
8      0.041339  0.015283      12        1   3.971214
18     0.027370  0.007489      13        1   3.460856
9     -0.151073 -0.037704      14        1   2.277022
15    -0.020293 -0.011551      15        1   2.247358
14    -0.121913 -0.034557      16        1   1.872036
0     -0.129250 -0.035449      17        1   1.730805
16    -0.191429 -0.040288      18        1   1.074433
11    -0.252171 -0.031504      19        1   0.048429
4     -0.253473 -0.031138      20        1   0.022112, topic_info=               Term          Freq         Total Category  logprob  loglift
5             covid  93677.000000  93677.000000  Default  30.0000  30.0000
58          vaccine  25186.000000  25186.000000  Default  29.0000  29.0000
0             death   9059.000000   9059.000000  Default  28.0000  28.0000
47              say  10707.000000  10707.000000  Default  27.0000  27.0000
50              get   6800.000000   6800.000000  Default  26.0000  26.0000
..              ...           ...           ...      ...      ...      ...
25  hospitalization      0.006769   1154.080200  Topic20  -9.7432  -3.6297
26            large      0.006769    722.000182  Topic20  -9.7432  -3.1606
27           reduce      0.006769   1255.509492  Topic20  -9.7432  -3.7139
28             risk      0.006769   4688.659358  Topic20  -9.7432  -5.0315
29       study_find      0.006769    263.046046  Topic20  -9.7432  -2.1509

[923 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
183       7  0.998914      accord
299       9  0.989229    activity
295      16  0.997328         add
214       8  0.993196  additional
338       7  0.997898      affect
...     ...       ...         ...
147       2  0.990478        wide
217      14  0.989336         win
110       4  0.998891       world
446       4  0.992378   worldwide
383       6  0.999674        year

[287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 2, 6, 4, 3, 8, 18, 7, 14, 20, 13, 9, 19, 10, 16, 15, 1, 17, 12, 5])